# VDEH Data Preprocessing Pipeline

**Fokus:** Datenbereinigung und Optimierung

## 🎯 Ziel
- Datentyp-Optimierung (Memory-Effizienz)
- ISBN-Spalte entfernen (leer bei VDEH)
- Kategorische Variablen optimieren
- Datenvalidierung und -bereinigung

## 📚 Input/Output
- **Input**: `data/vdeh/processed/01_loaded_data.parquet`
- **Output**: `data/vdeh/processed/02_preprocessed_data.parquet`

In [15]:
# 🛠️ SETUP UND DATEN LADEN
import sys
import pandas as pd
import numpy as np
from pathlib import Path
import json

# Projektroot finden
current_dir = Path.cwd()
project_root = None

for parent in [current_dir] + list(current_dir.parents):
    if (parent / 'config.yaml').exists():
        project_root = parent
        break

if project_root is None:
    raise FileNotFoundError("config.yaml nicht gefunden!")

# Config laden
src_path = project_root / 'src'
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

from config_loader import load_config
config = load_config(project_root / 'config.yaml')

print(f"📁 Projektroot: {project_root}")
print("✅ Konfiguration geladen")

✅ Konfiguration geladen: /media/sz/Data/Bibo/analysis/config.yaml
📁 Projektroot: /media/sz/Data/Bibo/analysis
✅ Konfiguration geladen


In [16]:
# 📂 DATEN AUS VORHERIGER STUFE LADEN
processed_dir = config.project_root / config.get('paths.data.vdeh.processed')
input_path = processed_dir / '01_loaded_data.parquet'
metadata_path = processed_dir / '01_metadata.json'

if not input_path.exists():
    raise FileNotFoundError(f"Input-Datei nicht gefunden: {input_path}\n"
                          "Bitte führen Sie zuerst 01_vdeh_data_loading.ipynb aus.")

# Daten laden
df_vdeh = pd.read_parquet(input_path)

# Vorherige Metadaten laden
with open(metadata_path, 'r') as f:
    prev_metadata = json.load(f)

print(f"📂 Daten geladen aus: {input_path}")
print(f"📊 Records: {len(df_vdeh):,}")
print(f"📋 Spalten: {list(df_vdeh.columns)}")
print(f"📅 Vorherige Verarbeitung: {prev_metadata['processing_date']}")
print(f"💾 Memory vor Optimierung: {df_vdeh.memory_usage(deep=True).sum() / 1024**2:.1f} MB")

📂 Daten geladen aus: /media/sz/Data/Bibo/analysis/data/vdeh/processed/01_loaded_data.parquet
📊 Records: 58,760
📋 Spalten: ['id', 'title', 'authors', 'authors_affiliation', 'year', 'publisher', 'isbn', 'issn', 'authors_str', 'num_authors', 'authors_affiliation_str', 'num_authors_affiliation']
📅 Vorherige Verarbeitung: 2025-11-06T12:48:38.378116
💾 Memory vor Optimierung: 37.4 MB
💾 Memory vor Optimierung: 37.4 MB


In [17]:
# 📋 ISBN/ISSN VALIDIERUNG
print("📋 === ISBN/ISSN VALIDIERUNG ===")

import re

def validate_isbn(isbn_str):
    """
    Validiert ISBN-10 oder ISBN-13 Format.
    Akzeptiert Formate mit und ohne Bindestriche.
    """
    if pd.isna(isbn_str):
        return None, None
    
    # Entferne alle Bindestriche und Leerzeichen
    isbn_clean = str(isbn_str).replace('-', '').replace(' ', '').upper()
    
    # ISBN-10: 10 Zeichen (letztes kann X sein)
    if len(isbn_clean) == 10:
        if not re.match(r'^\d{9}[\dX]$', isbn_clean):
            return False, "ISBN-10: Ungültiges Format"
        
        # Prüfziffer validieren
        try:
            checksum = sum((10 - i) * (10 if c == 'X' else int(c)) for i, c in enumerate(isbn_clean))
            is_valid = checksum % 11 == 0
            return is_valid, "ISBN-10" if is_valid else "ISBN-10: Ungültige Prüfziffer"
        except:
            return False, "ISBN-10: Validierungsfehler"
    
    # ISBN-13: 13 Zeichen
    elif len(isbn_clean) == 13:
        if not re.match(r'^\d{13}$', isbn_clean):
            return False, "ISBN-13: Ungültiges Format"
        
        # Prüfziffer validieren
        try:
            checksum = sum(int(c) * (1 if i % 2 == 0 else 3) for i, c in enumerate(isbn_clean))
            is_valid = checksum % 10 == 0
            return is_valid, "ISBN-13" if is_valid else "ISBN-13: Ungültige Prüfziffer"
        except:
            return False, "ISBN-13: Validierungsfehler"
    
    else:
        return False, f"ISBN: Ungültige Länge ({len(isbn_clean)} Zeichen)"

def validate_issn(issn_str):
    """
    Validiert ISSN Format (8 Zeichen: XXXX-XXXX).
    """
    if pd.isna(issn_str):
        return None, None
    
    # Entferne alle Bindestriche und Leerzeichen
    issn_clean = str(issn_str).replace('-', '').replace(' ', '').upper()
    
    # ISSN muss genau 8 Zeichen haben
    if len(issn_clean) != 8:
        return False, f"ISSN: Ungültige Länge ({len(issn_clean)} Zeichen)"
    
    # Format: 7 Ziffern + Prüfziffer (kann X sein)
    if not re.match(r'^\d{7}[\dX]$', issn_clean):
        return False, "ISSN: Ungültiges Format"
    
    # Prüfziffer validieren
    try:
        checksum = sum((8 - i) * int(c) if c != 'X' else 10 for i, c in enumerate(issn_clean[:7]))
        check_digit = (11 - (checksum % 11)) % 11
        expected = 'X' if check_digit == 10 else str(check_digit)
        is_valid = issn_clean[7] == expected
        return is_valid, "ISSN" if is_valid else f"ISSN: Ungültige Prüfziffer (erwartet: {expected})"
    except:
        return False, "ISSN: Validierungsfehler"

# ISBN Validierung (falls vorhanden)
if 'isbn' in df_vdeh.columns:
    print("\n📚 ISBN Validierung:")
    df_vdeh[['isbn_valid', 'isbn_status']] = df_vdeh['isbn'].apply(
        lambda x: pd.Series(validate_isbn(x))
    )
    
    isbn_with_values = df_vdeh['isbn'].notna().sum()
    if isbn_with_values > 0:
        valid_count = df_vdeh['isbn_valid'].sum()
        invalid_count = (df_vdeh['isbn_valid'] == False).sum()
        
        print(f"   Gesamt ISBNs: {isbn_with_values:,}")
        print(f"   ✅ Gültig: {valid_count:,} ({valid_count/isbn_with_values*100:.1f}%)")
        print(f"   ❌ Ungültig: {invalid_count:,} ({invalid_count/isbn_with_values*100:.1f}%)")
        
        # Beispiele ungültiger ISBNs
        if invalid_count > 0:
            print("\n   📋 Beispiele ungültiger ISBNs:")
            invalid_sample = df_vdeh[df_vdeh['isbn_valid'] == False][['isbn', 'isbn_status']].head(3)
            for _, row in invalid_sample.iterrows():
                print(f"      {row['isbn']} → {row['isbn_status']}")
    else:
        print("   ℹ️  Keine ISBN-Werte vorhanden")
else:
    print("\n📚 ISBN: Spalte nicht vorhanden")

# ISSN Validierung
if 'issn' in df_vdeh.columns:
    print("\n📰 ISSN Validierung:")
    df_vdeh[['issn_valid', 'issn_status']] = df_vdeh['issn'].apply(
        lambda x: pd.Series(validate_issn(x))
    )
    
    issn_with_values = df_vdeh['issn'].notna().sum()
    if issn_with_values > 0:
        valid_count = df_vdeh['issn_valid'].sum()
        invalid_count = (df_vdeh['issn_valid'] == False).sum()
        
        print(f"   Gesamt ISSNs: {issn_with_values:,}")
        print(f"   ✅ Gültig: {valid_count:,} ({valid_count/issn_with_values*100:.1f}%)")
        print(f"   ❌ Ungültig: {invalid_count:,} ({invalid_count/issn_with_values*100:.1f}%)")
        
        # Beispiele ungültiger ISSNs
        if invalid_count > 0:
            print("\n   📋 Beispiele ungültiger ISSNs:")
            invalid_sample = df_vdeh[df_vdeh['issn_valid'] == False][['issn', 'issn_status']].head(5)
            for _, row in invalid_sample.iterrows():
                print(f"      {row['issn']} → {row['issn_status']}")
        
        # ISSN-Typen Übersicht
        if valid_count > 0:
            print("\n   📊 Status-Übersicht:")
            status_counts = df_vdeh['issn_status'].value_counts()
            for status, count in status_counts.head(10).items():
                if pd.notna(status):
                    print(f"      {status}: {count:,} ({count/issn_with_values*100:.1f}%)")
    else:
        print("   ℹ️  Keine ISSN-Werte vorhanden")
else:
    print("\n📰 ISSN: Spalte nicht vorhanden")

# Zusammenfassung
print("\n" + "="*60)
print("💡 Hinweis: Ungültige Nummern können durch Tippfehler,")
print("   fehlerhafte OCR-Erkennung oder Datenfehler entstehen.")
print("="*60)

📋 === ISBN/ISSN VALIDIERUNG ===

📚 ISBN Validierung:
   Gesamt ISBNs: 11,415
   ✅ Gültig: 10,350 (90.7%)
   ❌ Ungültig: 1,065 (9.3%)

   📋 Beispiele ungültiger ISBNs:
      387011839 → ISBN: Ungültige Länge (9 Zeichen)
      3-885-08990-8 → ISBN-10: Ungültige Prüfziffer
      0-895-20419-6 → ISBN-10: Ungültige Prüfziffer

📰 ISSN Validierung:
   Gesamt ISBNs: 11,415
   ✅ Gültig: 10,350 (90.7%)
   ❌ Ungültig: 1,065 (9.3%)

   📋 Beispiele ungültiger ISBNs:
      387011839 → ISBN: Ungültige Länge (9 Zeichen)
      3-885-08990-8 → ISBN-10: Ungültige Prüfziffer
      0-895-20419-6 → ISBN-10: Ungültige Prüfziffer

📰 ISSN Validierung:
   Gesamt ISSNs: 721
   ✅ Gültig: 716 (99.3%)
   ❌ Ungültig: 5 (0.7%)

   📋 Beispiele ungültiger ISSNs:
      0952-2110 → ISSN: Ungültige Prüfziffer (erwartet: 5)
      0016-9733 → ISSN: Ungültige Prüfziffer (erwartet: 0)
      0304-3834 → ISSN: Ungültige Prüfziffer (erwartet: 5)
      3876-7796 → ISSN: Ungültige Prüfziffer (erwartet: 1)
      1984-9899 → ISSN: U

In [18]:
# 🔢 DATENTYP-OPTIMIERUNG
print("🔢 === DATENTYP-OPTIMIERUNG ===")

# Jahr-Spalte optimieren
year_dtype = config.get('data_processing.dataframe.year_dtype', 'Int64')
print(f"🔧 Konvertiere Year-Spalte zu {year_dtype}...")
df_vdeh['year'] = df_vdeh['year'].astype(year_dtype)
print(f"✅ Year-Spalte optimiert: {df_vdeh['year'].dtype}")

🔢 === DATENTYP-OPTIMIERUNG ===
🔧 Konvertiere Year-Spalte zu Int64...
✅ Year-Spalte optimiert: Int64


In [19]:
# 🔍 DATENVALIDIERUNG
print("🔍 === DATENVALIDIERUNG ===")

# Basis-Validierungen
print(f"📊 Gesamt Records: {len(df_vdeh):,}")
print(f"📋 Finale Spalten: {len(df_vdeh.columns)}")

# Prüfe kritische Felder
critical_fields = ['id', 'title', 'authors_str']
print("\n📋 Kritische Felder:")
for field in critical_fields:
    if field in df_vdeh.columns:
        null_count = df_vdeh[field].isnull().sum()
        null_pct = null_count / len(df_vdeh) * 100
        print(f"   {field:12}: {null_count:7,} null ({null_pct:5.1f}%)")
    else:
        print(f"   {field:12}: ⚠️ Spalte fehlt!")

# Jahr-Bereich prüfen
if 'year' in df_vdeh.columns:
    year_stats = df_vdeh['year'].describe()
    print(f"\n📅 Jahr-Statistiken:")
    print(f"   Min: {year_stats['min']}, Max: {year_stats['max']}")
    print(f"   Median: {year_stats['50%']}, Null: {df_vdeh['year'].isnull().sum():,}")

print(f"\n💾 Finale Datentypen:")
for col, dtype in df_vdeh.dtypes.items():
    print(f"   {col:15}: {dtype}")

🔍 === DATENVALIDIERUNG ===
📊 Gesamt Records: 58,760
📋 Finale Spalten: 16

📋 Kritische Felder:
   id          :       0 null (  0.0%)
   title       :  17,930 null ( 30.5%)
   authors_str :       0 null (  0.0%)

📅 Jahr-Statistiken:
   Min: 1900.0, Max: 2017.0
   Median: 1998.0, Null: 25,073

💾 Finale Datentypen:
   id             : object
   title          : object
   authors        : object
   authors_affiliation: object
   year           : Int64
   publisher      : object
   isbn           : object
   issn           : object
   authors_str    : object
   num_authors    : int64
   authors_affiliation_str: object
   num_authors_affiliation: int64
   isbn_valid     : object
   isbn_status    : object
   issn_valid     : object
   issn_status    : object


In [20]:
# 💾 PREPROCESSED DATEN EXPORTIEREN
output_path = processed_dir / '02_preprocessed_data.parquet'
df_vdeh.to_parquet(output_path, index=False)

print(f"💾 === DATA PREPROCESSING ABGESCHLOSSEN ===")
print(f"✅ Preprocessed Daten exportiert: {output_path}")
print(f"📊 Records: {len(df_vdeh):,}")
print(f"📋 Spalten: {len(df_vdeh.columns)}")
print(f"💾 Finale Dateigröße: {df_vdeh.memory_usage(deep=True).sum() / 1024**2:.1f} MB")

# Metadaten für nächste Stufe
metadata = {
    'stage': '02_data_preprocessing',
    'records_processed': len(df_vdeh),
    'columns': list(df_vdeh.columns),
    'data_types': {col: str(dtype) for col, dtype in df_vdeh.dtypes.items()},
    'memory_mb': df_vdeh.memory_usage(deep=True).sum() / 1024**2,
    'previous_stage': prev_metadata,
    'processing_date': pd.Timestamp.now().isoformat()
}

import json
metadata_path = processed_dir / '02_metadata.json'
with open(metadata_path, 'w') as f:
    json.dump(metadata, f, indent=2)

print(f"📄 Metadaten gespeichert: {metadata_path}")
print(f"\n➡️  Nächster Schritt: 03_vdeh_language_detection.ipynb")

💾 === DATA PREPROCESSING ABGESCHLOSSEN ===
✅ Preprocessed Daten exportiert: /media/sz/Data/Bibo/analysis/data/vdeh/processed/02_preprocessed_data.parquet
📊 Records: 58,760
📋 Spalten: 16
💾 Finale Dateigröße: 46.3 MB
📄 Metadaten gespeichert: /media/sz/Data/Bibo/analysis/data/vdeh/processed/02_metadata.json

➡️  Nächster Schritt: 03_vdeh_language_detection.ipynb
💾 Finale Dateigröße: 46.3 MB
📄 Metadaten gespeichert: /media/sz/Data/Bibo/analysis/data/vdeh/processed/02_metadata.json

➡️  Nächster Schritt: 03_vdeh_language_detection.ipynb
